# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
# !pip install requests
# !pip install speechrecognition==3.8.1
# !pip install paramiko
# !pip install pandas
# !pip install beautifulsoup4
# !pip install pocketsphinx
# !pip install scipy

In [3]:
from traceback import format_exc
from time      import sleep

from idle import idling

from interactiveControls import EyesController
from interactiveControls import track_head
from interactiveControls import return_to_default_pos

from humanInteraction import detect
from humanInteraction import listen
from humanInteraction import speak
from humanInteraction import faceTracker
from humanInteraction import processQuery
from humanInteraction import stopListening
from humanInteraction import promptForNextQuery
from humanInteraction import sayGoodbye

# from chatflow import convInit, convFlow

from behaviorManager import launchAndStopBehavior, service

RuntimeError: System error: Connection refused

## Constants

In [ ]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

# Choregraph Behavior

In [ ]:
### Guitar Behaviour ###
launchAndStopBehavior(service, '.lastUploadedChoregrapheBehavior/behavior_1')

## MAIN

In [ ]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = EyesController()
eyes.startEyes("neutral")

try:
    track_head()

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect(idle)
        faceTracker()
        # confusionCount = 0 # confusion level: number of times Pepper repeats herself and requests clarification
        state = {"topic": "fresh", "confusion": 0, "repeat": True, "confident": False}
        inConversation = True
        queryCount = 0

        while inConversation:

            while state["repeat"]:
                idle.stop()
                query = listen(eyes)
            
                state = processQuery(query, responsesPipeline, eyes, state)
                if state["confusion"] > 3:
                    state["repeat"] = False
                    inConversation = False
                    raise Exception("tooHot")
            
            # if not repeat, pause to read the tablet?
            sleep(5)

            queryCount += 1 
            if queryCount > 3:
                inConversation = False
                sayGoodbye()
                break
            elif state["topic"] != "Cour":
                promptForNextQuery() # Pepper asks if there are any other questions
                # prompt is not needed for "Cour", is embedded in the text         
                
            # then set repeat to True to enter the next While loop
            state["repeat"] = True
            # conversation reloops

        # after conversation ends, wait for a bit to reset 
        sleep(2)
        return_to_default_pos()
        sleep(10)
        
        continue
except Exception as e:
    print(e)
    print(format_exc())
finally:
    stopListening()
    return_to_default_pos()
    idle.start_idle_behavior()
    print('Goodbye')